In [3]:
import requests
import pandas as pd
import nltk
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

main_url = 'https://www.ccu.edu/undergrad/'

browser.visit(main_url)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\coled\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [4]:
html = browser.html

In [5]:
program_soup = bs(html, 'lxml')

In [6]:
program_urls = program_soup.select('li a.uk-panel')

In [7]:
program_urls

[<a class="uk-panel" href="accounting/"><span class="uk-margin-right">Accounting</span> <span class="uk-badge uk-badge-notification uk-float-right uk-margin-small-left">m</span> <span class="uk-badge uk-badge-warning uk-badge-notification uk-float-right uk-margin-small-left">M</span></a>,
 <a class="uk-panel" href="honors-liberal-arts/"><span class="uk-margin-right">Augustine Honors</span> <span class="uk-badge uk-badge-danger uk-badge-notification uk-float-right uk-margin-small-left">E</span> <span class="uk-badge uk-badge-notification uk-float-right uk-margin-small-left">m</span></a>,
 <a class="uk-panel" href="biblical-studies/"><span class="uk-margin-right">Biblical Studies with Languages</span> <span class="uk-badge uk-badge-success uk-badge-notification uk-float-right uk-margin-small-left">D</span> <span class="uk-badge uk-badge-notification uk-float-right uk-margin-small-left">m</span> <span class="uk-badge uk-badge-warning uk-badge-notification uk-float-right uk-margin-small-le

In [12]:
list_urls = []

for u in program_urls:
    program_link = u.get('href')
    program_url_full = f'https://www.ccu.edu/undergrad/{program_link}'
    browser.visit(program_url_full)
    page_html = browser.html
    page_soup = bs(html, 'html.parser')
    page_title = page_soup.find('title').text
    
    

In [ ]:
page_h1 = page_soup.find('h1').text
page_h1

In [ ]:
page_paras = page_soup.select('p')
page_paras

page_text = []

for p in page_paras:
    page_text.append(p.text)
    
page_text

In [ ]:
from cleantext import clean

In [ ]:
clean_paras = clean(page_text, no_punct=True)

In [ ]:
clean_paras

In [ ]:
tokens = [t for t in clean_paras.split()]

In [ ]:
tokens

In [ ]:
from nltk.corpus import stopwords

In [ ]:
sr= stopwords.words('english')

In [ ]:
clean_tokens = tokens[:]

In [ ]:
for token in tokens:
    if token in stopwords.words('english'):
        clean_tokens.remove(token)

In [ ]:
clean_tokens

In [ ]:
len(tokens)

In [ ]:
len(clean_tokens)

In [ ]:
freq = nltk.FreqDist(clean_tokens)

In [ ]:
freq.most_common(10)

In [ ]:
freq_words = []
for key,val in freq.most_common(10):
    words_dict = {}
    words_dict["word"] = key
    words_dict["count"] = val
    freq_words.append(words_dict)

In [ ]:
freq_words

In [ ]:
clean_tokens = tokens[:]

In [ ]:
for token in tokens:
    if token in stopwords.words('english'):
        clean_tokens.remove(token)

In [ ]:
clean_tokens

In [ ]:
for p in page_text:
    clean(page_text, no_punct=True)
    print(p)

In [ ]:
page_info = {}

In [ ]:
page_info["page_title"] = page_title
page_info["page_h1"] = page_h1
page_info["page_text"] = page_text

In [ ]:
page_info

In [ ]:
from cleantext import clean

In [ ]:
p_text = clean(p.text, no_punct=True)

In [ ]:
p_text

In [ ]:
tokens = [t for t in p_text.split()]

In [ ]:
token_text = []

tokens

In [ ]:
clean_page_text = []

for p in page_text:
    p_text = clean(p.text, no_punct=True)
    tokens = [t for t in p_text.split()]
    clean_page_text.append(tokens)

In [ ]:
browser.quit()